In [ ]:
## Preliminary notebook to learn about API and Python integration

In [12]:
import sys
sys.path.append("/home/natalie/anaconda3/envs/insight/lib/python3.6/site-packages")
from twython import Twython
import json

In [ ]:
Save twitter specifics to a json file

In [8]:
credentials = {}  
credentials['CONSUMER_KEY'] = '4gd8qFAcXZVvOtipmFwuGVV5v'  
credentials['CONSUMER_SECRET'] = 'R3gupzeBTifTXJUvpMDGTs61nfwpEd4EvlTB68b5lHZA6VYGML' 
credentials['ACCESS_TOKEN'] = '2882971510-d3OV32HpxWNVnC8XI87qS4uiaUISIolYOToui6N' 
credentials['ACCESS_SECRET'] = 'L12PhmS9uTl3sTw7L1G98PJmZOD4birfvc6EgPC3Z0uoE'

# Save the credentials object to file
with open("/home/natalie/Documents/Python_Training/twitterAPI/twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)


In [13]:
#initialize twitter

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

Search tweitter for the tweets occuring in the previous 7 days

In [29]:
# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query to find the most popular tweets containing the words 'learn python'
query = {'q': 'trump',  
        'result_type': 'mixed',
        'count': 40,
        'lang': 'en',
        
        }


Return a dictionary object of these results

In [30]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
df.sort_values(by='favorite_count', inplace=True, ascending=False)  
df.head(5)  

,user,date,text,favorite_count
0,SteveSchmidtSES,Wed Aug 22 05:35:11 +0000 2018,This means Trump’s Presidency is not only ille...,67235
2,JoeNBC,Tue Aug 21 21:44:32 +0000 2018,Republicans trying to spin this as not involvi...,49189
1,Acosta,Wed Aug 22 18:31:06 +0000 2018,Here is the cease and desist letter to Trump f...,26278
38,Sufiy,Thu Aug 23 15:26:17 +0000 2018,RT @business: How seriously should people take...,0
37,feefer2,Thu Aug 23 15:26:17 +0000 2018,"RT @TeamPelosi: Yes Paul, we will. \n\nhttps:/...",0


Tweets are only serachable for previous 7 days, no tweets older than that are shown in results. If we want to see the tweets moving forward in time we switch to the "streaming" version of this API.
https://stackabuse.com/accessing-the-twitter-api-with-python/

In [32]:
#streaming search

from twython import TwythonStreamer  
import csv

# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

Use the filter method to only collect tweets we're interested in. We'll create our filter with the track argument which provides the filter keywords, in our case "daycare".

In [34]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='daycare')  

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [ ]:
#data analysis with pandas

import pandas as pd  
tweets = pd.read_csv("saved_tweets.csv")  
tweets.head()  